In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
# Read the geopandas dataframe, from a parquet, containing the synthetic trajectories.
path_dataset = './dataset_simulator_atl_trajectories.parquet'
gdf = gpd.read_parquet(path_dataset)
gdf

In [ ]:
# Randomly select a certain number of users from the dataset. 
n_users = 5
list_users = np.random.choice(gdf['ID'].nunique(), n_users, replace=False).tolist()

users = gdf.loc[gdf['ID'].isin(list_users), :]
print(users.head())

In [ ]:
# Plot how the users move over time.

import folium
from folium.plugins import TimestampedGeoJson


# 2. Pick a palette of colors and map each unique ID to one
palette = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightred", "beige", "darkblue",
    "darkgreen", "cadetblue", "darkpurple"
]
unique_ids = users["ID"].unique()
color_map = {
    uid: palette[i % len(palette)]
    for i, uid in enumerate(unique_ids)
}


# 2. Build a GeoJSON-style dict with times
features = []
for _, row in users.iterrows():
    cid = row["ID"]
    features.append({
        "type": "Feature",
        "geometry": row.geometry.__geo_interface__,
        "properties": {
            "time": row.timestamp.isoformat(),
            "icon": "circle",
            "iconstyle": {
                "fillColor": color_map[cid],
                "fillOpacity": 0.7,
                "stroke": False,
                "radius": 10
            },
            "popup": f"ID {cid}<br>{row.timestamp:%Y-%m-%d %H:%M}"
        }
    })

geojson = {"type": "FeatureCollection", "features": features}


# 3. Create the map
m = folium.Map(location=[users.geometry.y.mean(), users.geometry.x.mean()], zoom_start=13, control_scale=True)

# 4. Add the time‐slider layer
duration_step_minutes = 5
TimestampedGeoJson(
    data=geojson,
    transition_time=200,      # milliseconds between frames
    period=f"PT{duration_step_minutes}M",            # matches your n-minutes stepping
    duration=f"PT{duration_step_minutes}M",
    add_last_point=False,
    auto_play=False,
    loop=False,
).add_to(m)


# 5. Display or save
m
# m.save("trajectory_time_slider.html")